# Final Project for AI 221

AI 221 | 1st Sem AY 2024-2025  
Students: Jemima Bian Anila, Joshua Cantor, Michael Spencer Quinto, MEng in AI  
Instructors: Dr. Miguel Remolona | Dr. Jon Dewitt Dalisay

## Dataset

https://data.mendeley.com/datasets/tsvdyhbphs/1

https://www.kaggle.com/datasets/jaydepaolomirandilla/philippine-medicinal-plant-leaf-dataset/data


In [1]:
import sys
from pathlib import Path
  
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import warnings
import torch  
import cv2
import random



%load_ext autoreload
%autoreload 2

pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
warnings.filterwarnings('ignore')
np.set_printoptions(suppress=True, precision=3)

/tmp/ipykernel_3153349/1189830066.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Add project root directory to Python path to allow importing from src folder
notebook_dir = Path("__file__").resolve().parent
project_root = Path.cwd().parent
sys.path.append(str(project_root))

In [3]:
print(torch.cuda.is_available())

True


# Import utility modules from `src` folder

Note: all reusable code like transformations for the dataframes, etc. can be found inside the `src` folder and are imported here for better modularity and version control


In [10]:
from src.utils_nn import SampleNN, test_function
from src.utils_data import (
    DATA_FOLDER,
    PROCESSED_DATA_FOLDER,
    HIBISCUS_PROCESSED_FOLDER,
    CARMONA_RETUSA_PROCESSED_FOLDER,
    AVERRHOEA_BILIMBI_PROCESSED_FOLDER,
)

In [5]:
print(PROCESSED_DATA_FOLDER.relative_to(project_root))
print(HIBISCUS_PROCESSED_FOLDER.relative_to(project_root))

data/ph_med_plants_reduced_sizes
data/ph_med_plants_reduced_sizes/1Hibiscus rosa-sinensis(HRS)


In [6]:
from src.utils_preprocessing import create_bg_rem_mask, remove_background

In [20]:
from tqdm import tqdm

BG_REMOVED_DATA_FOLDER = DATA_FOLDER / "bg_removed"

# Create the base directory if it doesn't exist
BG_REMOVED_DATA_FOLDER.mkdir(exist_ok=True)

# iter = 0

# Loop through all class folders
for class_folder in tqdm(
    list(PROCESSED_DATA_FOLDER.iterdir()), desc="Processing classes"
):
    if not class_folder.is_dir():
        continue

    # if iter > 1:
    #     break

    # Create corresponding folder in bg_removed directory
    bg_removed_class_folder = BG_REMOVED_DATA_FOLDER / class_folder.name
    bg_removed_class_folder.mkdir(exist_ok=True)

    # Process each image in the class folder

    # itter2 = 0

    for img_path in tqdm(
        list(class_folder.glob("*.webp")),
        desc=f"Processing {class_folder.name}",
        leave=False,
    ):
        # if itter2 > 1:
        #     break

        # Read image
        img = plt.imread(str(img_path))
        if img is None:
            print(f"Failed to read {img_path}")
            continue

        # Remove background
        mask = create_bg_rem_mask(img)
        bg_removed = remove_background(img, mask)

        # # Plot the background removed image using matplotlib
        # plt.figure(figsize=(10, 10))
        # plt.imshow(bg_removed)
        # plt.axis("off")
        # plt.title(f"Background Removed: {img_path.name}")
        # plt.show()

        # # Convert to BGR for saving
        # bg_removed = cv2.cvtColor(bg_removed, cv2.COLOR_RGB2BGR)

        # Save with same filename in bg_removed folder
        output_path = bg_removed_class_folder / img_path.name
        cv2.imwrite(str(output_path), bg_removed)

        # itter2 += 1

    # iter += 1

print("Background removal completed for all classes!")

Processing classes: 100%|██████████| 40/40 [1:10:05<00:00, 105.13s/it]

Background removal completed for all classes!
